# Тема “Работа с данными в Pandas”

### Задание 5**
Вычисление метрик для мониторинга.

    суммарная выручка 
    кол-во покупок 
    средний чек 
    среднее кол-во товаров в покупке
    
Дополнительно добавим возможность фильтровать данные по различным параметрам. Это может быть полезно чтобы посмотреть как меняются продажи в двух группах, или в какой-то отдельной категории товаров.

На вход функции будет подаваться датафрейм с данными о продажах, словарь с фильтрами и период, за который нужно посчитать метрики.
Функция должна вернуть датафрейм, в индексах которого будут все даты из указанного периода отсортированные по возрастанию, а столбцы - метрики \
['revenue', 'number_purchases', 'average_check', 'average_number_items']. \
Формат данных столбцов - float, формат данных индекса - datetime64[ns].
Если в какие-то дни не было продаж, то нужно заполнить пропуск нулём.


In [ ]:
def calculate_sales_metrics(df, cost_name, date_name, sale_id_name, period, filters=None):
   """Вычисляет метрики по продажам.
   df - pd.DataFrame, датафрейм с данными. Пример
       pd.DataFrame(
           [[820, '2021-04-03', 1, 213]],
           columns=['cost', 'date', 'sale_id', 'shop_id']
       )
   cost_name - str, название столбца с стоимостью товара
   date_name - str, название столбца с датой покупки
   sale_id_name - str, название столбца с идентификатором покупки (в одной покупке может быть несколько товаров)
   period - dict, словарь с датами начала и конца периода пилота.
       Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}.
       Дата начала периода входит в полуинтервал, а дата окончания нет,
       то есть '2020-01-01' <= date < '2020-01-08'.
   filters - dict, словарь с фильтрами. Ключ - название поля, по которому фильтруем, значение - список значений,
       которые нужно оставить. Например, {'user_id': [111, 123, 943]}.
       Если None, то фильтровать не нужно.
   return - pd.DataFrame, в индексах все даты из указанного периода отсортированные по возрастанию,
       столбцы - метрики ['revenue', 'number_purchases', 'average_check', 'average_number_items'].
       Формат данных столбцов - float, формат данных индекса - datetime64[ns].
   """
   # YOUR_CODE_HERE


In [62]:
import pandas as pd

def metrics(df, cost_name, date_name, sale_id_name, period, filters=None):

    df[date_name] = pd.to_datetime(df[date_name])
    df = df.sort_values(by=date_name)
    data = df.loc[(df[date_name] >= pd.to_datetime(period['begin'])) & (df[date_name] < pd.to_datetime(period['end']))]

    if filters:
        for key, val in filters.items():
            data = data.query(f"{key} in {val}")

    stat = pd.DataFrame(data.groupby(date_name).agg({cost_name : ['sum', 'count'], sale_id_name : ['nunique']}))
    stat.columns = ['revenue', 'count', 'number_purchases']
    stat['average_check'] = stat['revenue'] / stat['number_purchases']
    stat['average_number_items'] = stat['count'] / stat['number_purchases']
    stat.index.name = None
    stat.drop("count", axis=1, inplace=True)

    return stat

In [63]:
data = pd.DataFrame(
            [[820, '2021-04-29', 1, 213],
             [100, '2021-04-29', 1, 213],
             [550, '2021-04-29', 2, 213],
             [600, '2021-04-30', 3, 113],
             [700, '2021-05-01', 4, 893]],
            columns=['cost', 'date', 'sale_id', 'shop_id']
        )

In [64]:
metrics(data, 'cost', 'date', 'sale_id', 
                        period = {'begin': '2021-04-29', 'end': '2021-05-03'},
                       filters={'sale_id': [1, 3]})

,revenue,number_purchases,average_check,average_number_items
2021-04-29,920,1,920.0,2.0
2021-04-30,600,1,600.0,1.0


In [65]:
metrics(data, 'cost', 'date', 'sale_id', 
                        period = {'begin': '2021-04-29', 'end': '2021-05-01'})

,revenue,number_purchases,average_check,average_number_items
2021-04-29,1470,2,735.0,1.5
2021-04-30,600,1,600.0,1.0
